In [ ]:
from grid.interface import *
#launch()

In [ ]:
vast.offers('cuda_max_good >= 11.0 & rentable & gpu_name == "RTX 2080 Ti" & num_gpus == 2', default=True)

In [ ]:
refresh(['fuzzy-sleeve', ])

In [ ]:
sync_up('./output/pavlov', 'boardlaw:output/pavlov')
sync_up('./output/experiments/bee', 'boardlaw:output/experiments/bee')

In [ ]:
from grid.eval import *
vits = pd.concat([
    vitals(3),
    vitals(4),
    vitals(5),
    vitals(7),
    vitals(9)], 0)

In [ ]:
v = vitals(6)

In [ ]:
vits = v.combine_first(vits)

In [ ]:
(pn.ggplot(data=vits.query('boardsize in (7, 9)'))
    + pn.geom_text(pn.aes(x='flops', y='μ', label='depth.astype(int)'), size=8)
    + pn.scale_x_continuous(trans='log10')
    + mpl_theme())

In [ ]:
plot_params(vits)

In [ ]:
snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['nickname'] = snaps.run.str.extract('.* (.*)', expand=False) + '.' + snaps.idx.astype(str)
snaps['params'] = params(snaps)
snaps = snaps.set_index('nickname')

In [ ]:
games, wins = load(boardsize, snaps.index)

In [ ]:
soln = activelo.solve(games, wins)

In [ ]:
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmax())

In [ ]:
plot_flops(snaps)

In [ ]:
runs.pandas().query('description == "bee/8"')